In [303]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [304]:
dataset = pd.read_csv('datasets/Train.csv')
test = pd.read_csv('datasets/Test.csv')
data = pd.concat([dataset, test], axis = 0, sort=False)
# submission = pd.read_csv('datasets/SampleSubmission.csv')
# test.head()
# submission.head()
data.head()

,ID,DATOP,FLTID,DEPSTN,ARRSTN,STD,STA,STATUS,AC,target
0,train_id_0,2016-01-03,TU 0712,CMN,TUN,2016-01-03 10:30:00,2016-01-03 12.55.00,ATA,TU 32AIMN,260.0
1,train_id_1,2016-01-13,TU 0757,MXP,TUN,2016-01-13 15:05:00,2016-01-13 16.55.00,ATA,TU 31BIMO,20.0
2,train_id_2,2016-01-16,TU 0214,TUN,IST,2016-01-16 04:10:00,2016-01-16 06.45.00,ATA,TU 32AIMN,0.0
3,train_id_3,2016-01-17,TU 0480,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17.00.00,ATA,TU 736IOK,0.0
4,train_id_4,2016-01-17,TU 0338,TUN,ALG,2016-01-17 14:30:00,2016-01-17 15.50.00,ATA,TU 320IMU,22.0


In [305]:
data.AC.value_counts().head()

TU 320IMW    5084
TU 320IMT    5017
TU 320IMU    5006
TU 320IMR    4997
TU 320IMV    4989
Name: AC, dtype: int64

In [306]:
#dataset.DEPSTN.value_counts()
stn_most = ['TUN', 'DJE', 'ORY', 'MIR', 'MRS', 'LYS', 'NCE', 'ALG', 'MXP', 'IST']

data.DEPSTN = data.DEPSTN.apply(lambda x: 'Other' if x in stn_most else x)
data.ARRSTN = data.ARRSTN.apply(lambda x: 'Other' if x in stn_most else x)

In [307]:
data.ARRSTN.value_counts().head(10)

Other    83404
FRA       1497
BRU       1480
CMN       1472
FCO       1376
TOE       1355
CDG       1270
JED       1247
GVA       1206
SFA       1013
Name: ARRSTN, dtype: int64

In [308]:
data.isnull().sum()

ID           0
DATOP        0
FLTID        0
DEPSTN       0
ARRSTN       0
STD          0
STA          0
STATUS       0
AC           0
target    9333
dtype: int64

### Preprocessing

In [309]:
data.STA = data.STA.str.replace(".", ":")

In [310]:
# X = X.drop(['DATOP', 'STD', 'STA'], axis = 1)
data.DATOP = pd.to_datetime(data.DATOP)
data.STD = pd.to_datetime(data.STD)
data.STA = pd.to_datetime(data.STA)
#data.head()

In [311]:
# split date in new columns
def split_date_col(d, col_name):
    d[col_name + '_year'] = d[col_name].dt.year
    d[col_name + '_month'] = d[col_name].dt.month
    d[col_name + '_day'] = d[col_name].dt.day
    d[col_name + '_h'] = d[col_name].dt.hour
    d[col_name + '_m'] = d[col_name].dt.minute
    #d[col_name + '_s'] = d[col_name].dt.second
    return d

#data['DATOP_year'] = data['DATOP'].dt.year
#data['DATOP_month'] = data['DATOP'].dt.month
#data['DATOP_day'] = data['DATOP'].dt.day

data = split_date_col(data, 'STD')
data = split_date_col(data, 'STA')

data = data.drop(['DATOP', 'STD', 'STA'], axis = 1)

In [312]:
# split AC column
data.head()
new_ac_cols = pd.DataFrame(data.AC.str.split(' ', 1).tolist(),
                                   columns = ['AC_country','AC_model'])
data['AC_country'] = new_ac_cols.AC_country
data['AC_model'] = new_ac_cols.AC_model

data = data.drop(['AC'], axis = 1)
data.head()

,ID,FLTID,DEPSTN,ARRSTN,STATUS,target,STD_year,STD_month,STD_day,STD_h,STD_m,STA_year,STA_month,STA_day,STA_h,STA_m,AC_country,AC_model
0,train_id_0,TU 0712,CMN,Other,ATA,260.0,2016,1,3,10,30,2016,1,3,12,55,TU,32AIMN
1,train_id_1,TU 0757,Other,Other,ATA,20.0,2016,1,13,15,5,2016,1,13,16,55,TU,31BIMO
2,train_id_2,TU 0214,Other,Other,ATA,0.0,2016,1,16,4,10,2016,1,16,6,45,TU,32AIMN
3,train_id_3,TU 0480,Other,NTE,ATA,0.0,2016,1,17,14,10,2016,1,17,17,0,TU,736IOK
4,train_id_4,TU 0338,Other,Other,ATA,22.0,2016,1,17,14,30,2016,1,17,15,50,TU,320IMU


In [314]:
data.ARRSTN.value_counts()
data.AC_country.value_counts()
data.AC_model.value_counts().head(10)

320IMU    5194
320IMW    5094
320IMV    5054
320IMT    5020
320IMR    4984
320IMS    4662
32AIML    4512
32AIMP    4461
31BIMQ    4416
32AIMG    4289
Name: AC_model, dtype: int64

In [315]:
#ac_model_most = ['320IMU', '320IMW', '320IMV', '320IMT', '320IMR', '320IMS', '32AIML', '32AIMP', '31BIMQ', '32AIMG']
#data.AC_model = data.AC_model.apply(lambda x: 'Other' if x in ac_model_most else x)

In [323]:
data['AC_model2'] = data.AC_model.str[3:]
data.AC_model = data.AC_model.str[:3]

In [324]:
data.head()

,ID,FLTID,DEPSTN,ARRSTN,STATUS,target,STD_year,STD_month,STD_day,STD_h,STD_m,STA_year,STA_month,STA_day,STA_h,STA_m,AC_country,AC_model,AC_model2
0,train_id_0,TU 0712,CMN,Other,ATA,260.0,2016,1,3,10,30,2016,1,3,12,55,TU,32A,
1,train_id_1,TU 0757,Other,Other,ATA,20.0,2016,1,13,15,5,2016,1,13,16,55,TU,31B,
2,train_id_2,TU 0214,Other,Other,ATA,0.0,2016,1,16,4,10,2016,1,16,6,45,TU,32A,
3,train_id_3,TU 0480,Other,NTE,ATA,0.0,2016,1,17,14,10,2016,1,17,17,0,TU,736,
4,train_id_4,TU 0338,Other,Other,ATA,22.0,2016,1,17,14,30,2016,1,17,15,50,TU,320,


In [325]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
encoder = LabelEncoder()
scaler = StandardScaler()
one_hot_encoder = OneHotEncoder()

In [326]:
# data = pd.read_csv('datasets/Train.csv')
y = data['target']
X = data.drop(['ID', 'FLTID', 'target'], axis = 1)
X.head()

,DEPSTN,ARRSTN,STATUS,STD_year,STD_month,STD_day,STD_h,STD_m,STA_year,STA_month,STA_day,STA_h,STA_m,AC_country,AC_model,AC_model2
0,CMN,Other,ATA,2016,1,3,10,30,2016,1,3,12,55,TU,32A,
1,Other,Other,ATA,2016,1,13,15,5,2016,1,13,16,55,TU,31B,
2,Other,Other,ATA,2016,1,16,4,10,2016,1,16,6,45,TU,32A,
3,Other,NTE,ATA,2016,1,17,14,10,2016,1,17,17,0,TU,736,
4,Other,Other,ATA,2016,1,17,14,30,2016,1,17,15,50,TU,320,


In [327]:
X.STD_year = X.STA_year

In [328]:
def get_dummies_col(d, col_name):
    new_col = pd.DataFrame(d[col_name])
    new_col = pd.get_dummies(new_col)
    d = d.drop([col_name], axis = 1)
    d = pd.concat([d, new_col], axis = 1)
    return d

X = get_dummies_col(X, 'ARRSTN')
X = get_dummies_col(X, 'STATUS')
X = get_dummies_col(X, 'DEPSTN')
X = get_dummies_col(X, 'AC_country')
X = get_dummies_col(X, 'AC_model')
X = get_dummies_col(X, 'AC_model2')
X.head()

,STD_year,STD_month,STD_day,STD_h,STD_m,STA_year,STA_month,STA_day,STA_h,STA_m,...,AC_model_343,AC_model_345,AC_model_733,AC_model_734,AC_model_736,AC_model_738,AC_model_AT7,AC_model_CR9,AC_model_M87,AC_model2_
0,2016,1,3,10,30,2016,1,3,12,55,...,0,0,0,0,0,0,0,0,0,1
1,2016,1,13,15,5,2016,1,13,16,55,...,0,0,0,0,0,0,0,0,0,1
2,2016,1,16,4,10,2016,1,16,6,45,...,0,0,0,0,0,0,0,0,0,1
3,2016,1,17,14,10,2016,1,17,17,0,...,0,0,0,0,1,0,0,0,0,1
4,2016,1,17,14,30,2016,1,17,15,50,...,0,0,0,0,0,0,0,0,0,1


In [329]:
# X = X.drop(['DATOP', 'STD', 'STA'], axis = 1)

In [330]:
#scaler = StandardScaler()
#X = scaler.fit_transform(X)

In [331]:
# for test
test_data = pd.DataFrame(X).iloc[dataset.shape[0]:,:]
test_data.shape , X.shape

((9333, 292), (117166, 292))

In [332]:
# for train
Xtrain_kbira = pd.DataFrame(X).iloc[:dataset.shape[0],:]
Xtest_kbira = pd.DataFrame(X).iloc[dataset.shape[0]:,:]
Xtrain_kbira.shape , Xtest_kbira.shape

((107833, 292), (9333, 292))

In [333]:
ykbira = dataset.target

### Train part

In [334]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(Xtrain_kbira, ykbira, test_size = 0.00001)

In [335]:
from sklearn.metrics import mean_squared_error
def evaluate(reg):
    y_pred = reg.predict(x_test)
    result = np.sqrt(mean_squared_error(y_pred, y_test))
    print(result)

In [287]:
# Decision tree
from sklearn.tree import DecisionTreeRegressor
desicion_tree = DecisionTreeRegressor()
desicion_tree.fit(x_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [288]:
evaluate(desicion_tree)

30.405591591021544


In [336]:
from sklearn.ensemble import RandomForestRegressor
random_forest = RandomForestRegressor(n_estimators=30)
random_forest.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [337]:
evaluate(random_forest)

111.18698215168895


In [291]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [292]:
evaluate(lr)

10.286520533514489


In [293]:
#from sklearn.svm import SVR
#svm_svr = SVR()
#svm_svr.fit(x_train, y_train)

In [294]:
#evaluate(svm_svr)

In [295]:
#from sklearn.neighbors import KNeighborsRegressor
#knn = KNeighborsRegressor(1)
#knn.fit(x_train, y_train)

In [296]:
#evaluate(knn)

### Test

In [338]:
test_data.head()

,STD_year,STD_month,STD_day,STD_h,STD_m,STA_year,STA_month,STA_day,STA_h,STA_m,...,AC_model_343,AC_model_345,AC_model_733,AC_model_734,AC_model_736,AC_model_738,AC_model_AT7,AC_model_CR9,AC_model_M87,AC_model2_
0,2016,5,4,6,40,2016,5,4,7,30,...,0,0,0,0,0,0,0,0,0,1
1,2016,5,5,15,20,2016,5,5,20,5,...,0,0,0,0,0,0,0,0,0,1
2,2016,5,6,10,0,2016,5,6,12,25,...,0,0,0,0,0,0,0,0,0,1
3,2016,5,11,9,40,2016,5,11,13,10,...,0,0,0,0,1,0,0,0,0,1
4,2016,5,11,9,50,2016,5,11,12,35,...,0,0,0,0,0,0,0,0,0,1


In [339]:
#pd.DataFrame(test_set).head()

In [340]:
prediction = random_forest.predict(Xtest_kbira)

In [341]:
result_columns_list = list(zip(test.ID, prediction))
result = pd.DataFrame(result_columns_list, columns = ['ID', 'target'])

In [342]:
result.head()

,ID,target
0,test_id_0,6.400000
1,test_id_1,25.633333
2,test_id_2,40.066667
3,test_id_3,21.966667
4,test_id_4,64.533333


In [343]:
# export result
result.to_csv('SubmissionFile_3.csv', index=False)